## Reikalingų bibliotekų importavimas
Importuokite PyTorch ir transformers bibliotekas, reikalingas Phi-4 modelio įkėlimui ir naudojimui.


In [ ]:
import torch
from transformers import AutoModelForCausalLM, AutoTokenizer, pipeline

## Nustatyti atsitiktinę sėklą
Nustatykite atsitiktinę sėklą, kad skirtingi paleidimai duotų pakartojamus rezultatus.


In [ ]:
torch.random.manual_seed(0)

## Įkelti Phi-4-mini-flash-reasoning modelį ir tokenizatorių
Įkelkite Microsoft Phi-4-mini-flash-reasoning modelį ir jo atitinkamą tokenizatorių iš Hugging Face. Modelis bus užkrautas į CUDA greitesniam prognozavimui.


In [ ]:
model_id = "microsoft/Phi-4-mini-flash-reasoning"
model = AutoModelForCausalLM.from_pretrained(
   model_id,
   device_map="cuda",
   torch_dtype="auto",
   trust_remote_code=True,
)
tokenizer = AutoTokenizer.from_pretrained(model_id)

## Paruošti įvesties pranešimą
Sukurkite pokalbio pranešimą su kvadratinės lygties matematikos uždaviniu ir suformatuokite jį, naudodami pokalbio šabloną modeliui.


In [ ]:
messages = [{
   "role": "user",
   "content": "How to solve 3*x^2+4*x+5=1?"
}]   
inputs = tokenizer.apply_chat_template(
   messages,
   add_generation_prompt=True,
   return_dict=True,
   return_tensors="pt",
)

## Generuoti atsakymą
Generuokite atsakymą iš modelio naudodami nurodytus parametrus, tokius kaip temperature ir top_p, kad kontroliuotumėte išvesties atsitiktinumą.


In [ ]:
outputs = model.generate(
   **inputs.to(model.device),
   max_new_tokens=32768,
   temperature=0.6,
   top_p=0.95,
   do_sample=True,
)

## Dekoduoti išvestį į tekstą
Konvertuokite sugeneruotas žetonų sekas atgal į žmogui suprantamą tekstą, neįtraukdami originalių įvesties žetonų, kad būtų parodytas tik modelio atsakymas.


In [ ]:
outputs = tokenizer.batch_decode(outputs[:, inputs["input_ids"].shape[-1]:])

---

<!-- CO-OP TRANSLATOR DISCLAIMER START -->
**Atsakomybės apribojimas**:
Šis dokumentas buvo išverstas pasitelkiant dirbtinio intelekto vertimo paslaugą [Co-op Translator](https://github.com/Azure/co-op-translator). Nors siekiame tikslumo, atkreipkite dėmesį, kad automatizuoti vertimai gali turėti klaidų ar netikslumų. Originalus dokumentas jo gimtąja kalba turėtų būti laikomas autoritetingu šaltiniu. Svarbios informacijos atveju rekomenduojamas profesionalus žmogaus atliktas vertimas. Mes neatsakome už jokių nesusipratimų ar neteisingų interpretacijų, kylančių dėl šio vertimo naudojimo.
<!-- CO-OP TRANSLATOR DISCLAIMER END -->
